In [1]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import warnings
import gc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
# import swifter
gc.enable()
warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize': (20, 20)})
student_id = 2000728661
sample_size = 10000

df = pd.read_csv("dataset/2021_Competition_Training.csv", low_memory=False)

tdf = pd.read_csv("dataset/2021_Competition_Holdout.csv", low_memory=False)



In [2]:
dtypes = {}
for i in df.columns:
    dtype = df[i].dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += 1
dtypes

{dtype('int64'): 51, dtype('O'): 115, dtype('float64'): 202}

In [3]:
target = "covid_vaccination"

reg_cols, cat_cols = [], []
id = "ID"

for i in df.columns:
	if i not in [id, target]:
		if df[i].nunique() > 100:
			reg_cols.append(i)
		else:
			cat_cols.append(i)

# reg_cols = ["rx_gpi2_72_pmpm_cost_6to9m_b4", "atlas_pct_laccess_child15", "atlas_recfacpth14", "atlas_pct_fmrkt_frveg16", "atlas_pct_free_lunch14", "atlas_pc_snapben15", "credit_bal_nonmtgcredit_60dpd", "rx_bh_mbr_resp_pmpm_cost_9to12m_b4", "atlas_pct_laccess_nhna15", "credit_hh_nonmtgcredit_60dpd", "cons_lwcm10", "atlas_fsrpth14", "atlas_wicspth12", "cmsd2_sns_digest_abdomen_pmpm_ct", "credit_hh_bankcardcredit_60dpd", "total_outpatient_allowed_pmpm_cost_6to9m_b4", "atlas_netmigrationrate1016", "atlas_pct_laccess_snap15", "rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4", "atlas_naturalchangerate1016", "ccsp_236_pct", "atlas_pct_laccess_hisp15", "rx_overall_mbr_resp_pmpm_cost", "atlas_pct_laccess_hhnv15", "credit_bal_consumerfinance", "rwjf_uninsured_pct", "rx_mail_mbr_resp_pmpm_cost_0to3m_b4", "atlas_pct_wic15", "ccsp_193_pct", "atlas_pct_fmrkt_baked16", "rx_nonmaint_mbr_resp_pmpm_cost", "credit_hh_bankcard_severederog", "rx_hum_16_pmpm_ct", "cnt_cp_webstatement_pmpm_ct", "atlas_pct_laccess_seniors15", "phy_em_px_pct", "atlas_percapitainc", "rwjf_uninsured_adults_pct", "rx_generic_mbr_resp_pmpm_cost_0to3m_b4", "rx_gpi2_02_pmpm_cost", "atlas_pct_sfsp15", "total_physician_office_net_paid_pmpm_cost_9to12m_b4", "atlas_pc_dirsales12", "cms_tot_partd_payment_amt", "rx_nonotc_dist_gpi6_pmpm_ct", "rx_nonmaint_pmpm_ct", "rx_nonbh_mbr_resp_pmpm_cost_6to9m_b4", "rx_nonbh_mbr_resp_pmpm_cost", "atlas_redemp_snaps16", "total_physician_office_copay_pmpm_cost", "atlas_pct_fmrkt_anmlprod16", "credit_num_agencyfirstmtg", "atlas_agritrsm_rct12", "atlas_pct_laccess_pop15", "rx_gpi2_01_pmpm_cost_0to3m_b4", "rwjf_uninsured_child_pct", "credit_bal_mtgcredit_new", "atlas_pct_laccess_nhasian15", "atlas_deep_pov_all", "atlas_net_international_migration_rate", "atlas_deep_pov_children", "bh_ncdm_pct", "rx_branded_mbr_resp_pmpm_cost", "atlas_pc_wic_redemp12", "rwjf_mv_deaths_rate",
#             "atlas_pct_reduced_lunch14", "rx_hum_28_pmpm_cost", "atlas_totalocchu", "atlas_pct_loclfarm12", "rx_generic_mbr_resp_pmpm_cost", "total_outpatient_mbr_resp_pmpm_cost_6to9m_b4", "rx_gpi4_3400_pmpm_ct", "lab_dist_loinc_pmpm_ct", "atlas_pct_nslp15", "atlas_pct_laccess_lowi15", "atlas_pct_fmrkt_sfmnp16", "atlas_pct_loclsale12", "credit_bal_autobank", "rx_overall_mbr_resp_pmpm_cost_0to3m_b4", "rx_nonbh_net_paid_pmpm_cost", "cms_risk_adjustment_factor_a_amt", "rx_generic_pmpm_cost", "credit_num_autofinance", "rx_maint_mbr_resp_pmpm_cost_6to9m_b4", "atlas_pct_laccess_black15", "atlas_hh65plusalonepct", "bh_outpatient_net_paid_pmpm_cost", "rx_generic_pmpm_cost_6to9m_b4", "atlas_convspth14", "total_med_allowed_pmpm_cost_9to12m_b4", "atlas_pc_ffrsales12", "credit_bal_bankcard_severederog", "rx_gpi2_34_pmpm_ct", "atlas_veg_acrespth12", "atlas_grocpth14", "atlas_pct_fmrkt_snap16", "met_obe_diag_pct", "cms_partd_ra_factor_amt", "atlas_pct_sbp15", "rwjf_resident_seg_black_inx", "atlas_pct_cacfp15", "pdc_lip", "atlas_ffrpth14", "credit_num_autobank_new", "rx_tier_2_pmpm_ct", "atlas_berry_acrespth12", "atlas_pct_fmrkt_credit16", "atlas_pc_fsrsales12", "credit_hh_1stmtgcredit", "atlas_pct_fmrkt_wiccash16", "atlas_fmrktpth16", "cci_dia_m_pmpm_ct", "rwjf_income_inequ_ratio", "credit_num_nonmtgcredit_60dpd", "credit_bal_autofinance_new", "rwjf_men_hlth_prov_ratio", "bh_ncal_pct", "atlas_pct_snap16", "ccsp_227_pct", "atlas_ghveg_sqftpth12", "atlas_orchard_acrespth12", "atlas_pct_laccess_multir15", "atlas_medhhinc", "rwjf_mental_distress_pct", "zip_cd", "atlas_pct_laccess_nhpi15", "credit_num_consumerfinance_new", "rx_gpi2_49_pmpm_cost_0to3m_b4", "rx_overall_net_paid_pmpm_cost_6to9m_b4", "atlas_ownhomepct", "atlas_pct_fmrkt_wic16", "rwjf_social_associate_rate", "mcc_end_pct", "cons_lwcm07", "atlas_pct_fmrkt_otherfood16", "atlas_pct_laccess_white15", "rx_gpi2_66_pmpm_ct"]
# cat_cols = ["auth_3mth_post_acute_dia", "bh_ip_snf_net_paid_pmpm_cost_9to12m_b4", "auth_3mth_acute_ckd", "bh_ncal_pmpm_ct", "src_div_id", "total_bh_copay_pmpm_cost_t_9-6-3m_b4", "bh_ip_snf_net_paid_pmpm_cost_3to6m_b4", "cons_chmi", "mcc_ano_pmpm_ct_t_9-6-3m_b4", "auth_3mth_post_acute_trm", "rx_maint_pmpm_cost_t_12-9-6m_b4", "auth_3mth_post_acute_rsk", "cons_ltmedicr", "rx_gpi4_6110_pmpm_ct", "rx_nonbh_pmpm_cost_t_9-6-3m_b4", "auth_3mth_acute_vco", "rx_bh_pmpm_ct_0to3m_b4", "auth_3mth_dc_ltac", "auth_3mth_post_acute_inj", "auth_3mth_dc_home", "rx_gpi2_17_pmpm_cost_t_12-9-6m_b4", "cons_hxmioc", "rx_generic_pmpm_cost_t_6-3-0m_b4", "atlas_ghveg_farms12", "cons_cwht", "bh_ncdm_ind", "atlas_retirement_destination_2015_upda", "rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4", "bh_ip_snf_mbr_resp_pmpm_cost_6to9m_b4", "rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4", "auth_3mth_post_acute_ben", "auth_3mth_dc_no_ref", "rx_overall_gpi_pmpm_ct_0to3m_b4", "auth_3mth_dc_snf", "rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4", "auth_3mth_acute_ccs_048", "bh_ip_snf_net_paid_pmpm_cost_0to3m_b4", "auth_3mth_acute_end", "auth_3mth_psychic", "atlas_hiamenity", "auth_3mth_bh_acute", "auth_3mth_acute_chf", "rx_overall_gpi_pmpm_ct_t_6-3-0m_b4", "mcc_chf_pmpm_ct_t_9-6-3m_b4", "bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4", "auth_3mth_hospice", "auth_3mth_acute_bld", "auth_3mth_dc_hospice", "auth_3mth_acute_ccs_030", "auth_3mth_acute_skn", "atlas_veg_farms12", "atlas_vlfoodsec_13_15", "rx_gpi2_34_dist_gpi6_pmpm_ct", "bh_ip_snf_net_paid_pmpm_cost", "est_age", "rx_maint_pmpm_cost_t_6-3-0m_b4", "auth_3mth_acute_neo", "rwjf_air_pollute_density", "atlas_recfac14", "cons_mobplus", "lab_albumin_loinc_pmpm_ct", "atlas_pct_obese_adults13", "rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4", "rev_pm_obsrm_pmpm_ct", "med_ip_snf_admit_days_pmpm", "rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4", "auth_3mth_post_acute_vco", "cons_stlnindx", "atlas_hipov_1115", "auth_3mth_post_acute_dig", "atlas_berry_farms12", "rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4", "rwjf_inactivity_pct", "rx_gpi2_72_pmpm_ct_6to9m_b4", "cons_n2pmr", "med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4", "auth_3mth_acute_res", "rev_cms_ct_pmpm_ct", "atlas_foodhub16", "auth_3mth_acute_dig", "auth_3mth_dc_acute_rehab", "auth_3mth_post_acute_hdz", "bh_ip_snf_mbr_resp_pmpm_cost_3to6m_b4", "auth_3mth_acute_ccs_172", "total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4", "auth_3mth_acute_ccs_154", "atlas_type_2015_mining_no", "rx_days_since_last_script", "auth_3mth_post_acute_res", "auth_3mth_acute_inf", "atlas_povertyallagespct", "rx_branded_pmpm_ct_t_6-3-0m_b4", "med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4", "atlas_low_employment_2015_update", "atlas_pct_diabetes_adults13", "auth_3mth_non_er", "atlas_foodinsec_child_03_11", "auth_3mth_acute_cad", "cons_nwperadult", "total_allowed_pmpm_cost_t_9-6-3m_b4", "mabh_seg", "cms_orig_reas_entitle_cd", "med_physician_office_ds_clm_6to9m_b4", "bh_ip_snf_mbr_resp_pmpm_cost_9to12m_b4", "auth_3mth_post_acute_cir", "auth_3mth_post_acute_cer", "rx_generic_pmpm_ct_0to3m_b4", "oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4", "bh_ncal_ind", "auth_3mth_post_acute_mus", "hum_region", "rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4", "bh_ip_snf_net_paid_pmpm_cost_6to9m_b4", "rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4",
#             "med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4", "rx_tier_2_pmpm_ct_3to6m_b4", "rx_maint_pmpm_ct_9to12m_b4", "rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4", "atlas_type_2015_recreation_no", "auth_3mth_post_acute_sns", "rx_gpi2_39_pmpm_cost_t_6-3-0m_b4", "atlas_type_2015_update", "total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4", "cmsd2_eye_retina_pmpm_ct", "auth_3mth_acute_can", "auth_3mth_post_acute", "auth_3mth_facility", "rx_days_since_last_script_0to3m_b4", "atlas_population_loss_2015_update", "rx_maint_pmpm_ct_t_6-3-0m_b4", "auth_3mth_post_acute_men", "auth_3mth_acute_mean_los", "cons_rxmaint", "rx_mail_net_paid_pmpm_cost_t_6-3-0m_b4", "auth_3mth_home", "cons_hxwearbl", "total_physician_office_mbr_resp_pmpm_cost_t_9-6-3m_b4", "atlas_farm_to_school13", "auth_3mth_acute_inj", "auth_3mth_acute_ccs_153", "rej_days_since_last_clm", "auth_3mth_transplant", "atlas_dirsales_farms12", "rev_cms_ansth_pmpm_ct", "rx_mail_mbr_resp_pmpm_cost_t_9-6-3m_b4", "med_outpatient_visit_ct_pmpm_t_12-9-6m_b4", "rx_nonbh_pmpm_ct_t_9-6-3m_b4", "auth_3mth_acute", "rx_nonbh_pmpm_ct_0to3m_b4", "auth_3mth_dc_left_ama", "atlas_povertyunder18pct", "rx_tier_1_pmpm_ct_0to3m_b4", "auth_3mth_acute_ccs_227", "cons_estinv30_rc", "auth_3mth_bh_acute_men", "auth_3mth_dc_custodial", "total_med_net_paid_pmpm_cost_t_6-3-0m_b4", "rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4", "atlas_csa12", "sex_cd", "rx_gpi2_62_pmpm_cost_t_9-6-3m_b4", "lang_spoken_cd", "rx_overall_gpi_pmpm_ct_t_12-9-6m_b4", "auth_3mth_ltac", "cons_hhcomp", "auth_3mth_acute_hdz", "cons_rxadhs", "auth_3mth_acute_men", "auth_3mth_rehab", "auth_3mth_acute_ccs_086", "cons_n2pwh", "rx_nonmaint_dist_gpi6_pmpm_ct_t_12-9-6m_b4", "atlas_slhouse12", "auth_3mth_snf_post_hsp", "atlas_foodinsec_13_15", "auth_3mth_acute_cer", "cons_rxadhm", "rx_nonotc_pmpm_cost_t_6-3-0m_b4", "auth_3mth_acute_trm", "cons_n2phi", "bh_physician_office_copay_pmpm_cost_6to9m_b4", "rej_total_physician_office_visit_ct_pmpm_0to3m_b4", "auth_3mth_acute_dia", "auth_3mth_snf_direct", "auth_3mth_acute_ccs_067", "auth_3mth_acute_ccs_043", "auth_3mth_dc_home_health", "rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4", "cmsd2_sns_genitourinary_pmpm_ct", "auth_3mth_acute_cir", "auth_3mth_acute_ner", "auth_3mth_acute_ccs_094", "med_ambulance_coins_pmpm_cost_t_9-6-3m_b4", "hedis_dia_hba1c_ge9", "rx_days_since_last_script_6to9m_b4", "atlas_persistentchildpoverty_1980_2011", "auth_3mth_post_acute_cad", "cons_cgqs", "ccsp_065_pmpm_ct", "auth_3mth_acute_ccs_044", "rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4", "bh_ip_snf_admit_days_pmpm_t_9-6-3m_b4", "rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4", "auth_3mth_post_acute_ckd", "auth_3mth_post_acute_ner", "auth_3mth_post_er", "atlas_avghhsize", "atlas_orchard_farms12", "total_physician_office_visit_ct_pmpm_t_6-3-0m_b4", "rx_gpi2_33_pmpm_ct_0to3m_b4", "auth_3mth_post_acute_chf", "atlas_freshveg_farms12", "auth_3mth_acute_ccs_042", "auth_3mth_post_acute_inf", "auth_3mth_acute_sns", "days_since_last_clm_0to3m_b4", "auth_3mth_dc_other", "auth_3mth_bh_acute_mean_los", "auth_3mth_post_acute_gus", "auth_3mth_post_acute_end", "auth_3mth_acute_mus", "atlas_perpov_1980_0711", "auth_3mth_post_acute_mean_los", "auth_3mth_acute_gus", "rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4", "atlas_low_education_2015_update", "race_cd"]

tdf['auth_3mth_post_acute_mean_los'] = np.int64(tdf['auth_3mth_post_acute_mean_los'])
reg_cols.append('auth_3mth_post_acute_mean_los')
cat_cols = list(set(cat_cols) - set(['auth_3mth_post_acute_mean_los']))
reg_cols = list(set(reg_cols) - set(['Unnamed: 0']))
print(df.shape)
print(df.columns)

(974842, 368)
Index(['Unnamed: 0', 'ID', 'auth_3mth_post_acute_dia',
       'rx_gpi2_72_pmpm_cost_6to9m_b4', 'atlas_pct_laccess_child15',
       'atlas_recfacpth14', 'atlas_pct_fmrkt_frveg16',
       'atlas_pct_free_lunch14', 'bh_ip_snf_net_paid_pmpm_cost_9to12m_b4',
       'auth_3mth_acute_ckd',
       ...
       'auth_3mth_post_acute_end', 'auth_3mth_acute_mus',
       'atlas_perpov_1980_0711', 'atlas_pct_laccess_white15',
       'auth_3mth_post_acute_mean_los', 'rx_gpi2_66_pmpm_ct',
       'auth_3mth_acute_gus', 'rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
       'atlas_low_education_2015_update', 'race_cd'],
      dtype='object', length=368)


In [4]:
dtypes = {}
isnull = set()
for i in cat_cols:
    dtype = df[i].dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += 1
    if df[i].isnull().sum() > 0:
        isnull.add((i, dtype))
    if dtype == 'float64': print(i, df[i].nunique())
dtypes, isnull

cons_hxwearbl 10
atlas_foodinsec_13_15 41
cons_rxadhm 10
atlas_type_2015_update 6
atlas_csa12 89
atlas_hipov_1115 2
atlas_type_2015_recreation_no 2
atlas_low_education_2015_update 2
cons_estinv30_rc 10
cons_n2phi 100
atlas_hiamenity 2
atlas_perpov_1980_0711 2
cons_stlnindx 10
atlas_ghveg_farms12 47
cons_n2pwh 100
atlas_farm_to_school13 2
atlas_low_employment_2015_update 2
atlas_foodhub16 5
atlas_persistentchildpoverty_1980_2011 2
atlas_type_2015_mining_no 2
atlas_foodinsec_child_03_11 34
cons_hxmioc 10
cms_orig_reas_entitle_cd 4
race_cd 7
atlas_population_loss_2015_update 2
cons_cwht 100
cons_rxmaint 10
cons_rxadhs 10
atlas_retirement_destination_2015_upda 2
atlas_vlfoodsec_13_15 29
cons_ltmedicr 10
cons_n2pmr 93
atlas_slhouse12 11


({dtype('O'): 113, dtype('int64'): 45, dtype('float64'): 33},
 {('atlas_csa12', dtype('float64')),
  ('atlas_farm_to_school13', dtype('float64')),
  ('atlas_foodhub16', dtype('float64')),
  ('atlas_foodinsec_13_15', dtype('float64')),
  ('atlas_foodinsec_child_03_11', dtype('float64')),
  ('atlas_ghveg_farms12', dtype('float64')),
  ('atlas_hiamenity', dtype('float64')),
  ('atlas_hipov_1115', dtype('float64')),
  ('atlas_low_education_2015_update', dtype('float64')),
  ('atlas_low_employment_2015_update', dtype('float64')),
  ('atlas_perpov_1980_0711', dtype('float64')),
  ('atlas_persistentchildpoverty_1980_2011', dtype('float64')),
  ('atlas_population_loss_2015_update', dtype('float64')),
  ('atlas_retirement_destination_2015_upda', dtype('float64')),
  ('atlas_slhouse12', dtype('float64')),
  ('atlas_type_2015_mining_no', dtype('float64')),
  ('atlas_type_2015_recreation_no', dtype('float64')),
  ('atlas_type_2015_update', dtype('float64')),
  ('atlas_vlfoodsec_13_15', dtype('floa

In [5]:
dtypes = {}
isnullt = set()
for i in cat_cols:
    dtype = tdf[i].dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += 1
    if tdf[i].isnull().sum() > 0:
        isnullt.add((i, dtype))
    if dtype == 'float64': print(i, tdf[i].nunique())
dtypes, isnullt, isnullt - isnull

cons_hxwearbl 10
atlas_foodinsec_13_15 41
cons_rxadhm 10
atlas_type_2015_update 6
atlas_csa12 87
atlas_hipov_1115 2
atlas_type_2015_recreation_no 2
atlas_low_education_2015_update 2
cons_estinv30_rc 10
cons_n2phi 100
atlas_hiamenity 2
atlas_perpov_1980_0711 2
cons_stlnindx 10
atlas_ghveg_farms12 47
cons_n2pwh 100
atlas_farm_to_school13 2
atlas_low_employment_2015_update 2
atlas_foodhub16 5
atlas_persistentchildpoverty_1980_2011 2
atlas_type_2015_mining_no 2
atlas_foodinsec_child_03_11 34
cons_hxmioc 10
cms_orig_reas_entitle_cd 4
race_cd 7
atlas_population_loss_2015_update 2
cons_cwht 100
cons_rxmaint 10
cons_rxadhs 10
atlas_retirement_destination_2015_upda 2
atlas_vlfoodsec_13_15 29
cons_ltmedicr 10
cons_n2pmr 93
atlas_slhouse12 11


({dtype('O'): 107, dtype('int64'): 51, dtype('float64'): 33},
 {('atlas_csa12', dtype('float64')),
  ('atlas_farm_to_school13', dtype('float64')),
  ('atlas_foodhub16', dtype('float64')),
  ('atlas_foodinsec_13_15', dtype('float64')),
  ('atlas_foodinsec_child_03_11', dtype('float64')),
  ('atlas_ghveg_farms12', dtype('float64')),
  ('atlas_hiamenity', dtype('float64')),
  ('atlas_hipov_1115', dtype('float64')),
  ('atlas_low_education_2015_update', dtype('float64')),
  ('atlas_low_employment_2015_update', dtype('float64')),
  ('atlas_perpov_1980_0711', dtype('float64')),
  ('atlas_persistentchildpoverty_1980_2011', dtype('float64')),
  ('atlas_population_loss_2015_update', dtype('float64')),
  ('atlas_retirement_destination_2015_upda', dtype('float64')),
  ('atlas_slhouse12', dtype('float64')),
  ('atlas_type_2015_mining_no', dtype('float64')),
  ('atlas_type_2015_recreation_no', dtype('float64')),
  ('atlas_type_2015_update', dtype('float64')),
  ('atlas_vlfoodsec_13_15', dtype('floa

In [6]:
dtypes = {}
isnull = set()
for i in reg_cols:
    dtype = df[i].dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += 1
    if df[i].isnull().sum() > 0:
        isnull.add((i, dtype))
    if dtype == 'float64': print(i, df[i].nunique())
dtypes, isnull

credit_bal_bankcard_severederog 970174
atlas_pc_fsrsales12 22552
atlas_pc_dirsales12 887065
rx_gpi2_01_pmpm_cost_0to3m_b4 9558
rx_gpi4_3400_pmpm_ct 1414
rwjf_mv_deaths_rate 880791
atlas_fmrktpth16 799518
rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4 53518
rx_generic_pmpm_ct_0to3m_b4 237
atlas_orchard_acrespth12 765155
atlas_redemp_snaps16 890669
rev_pm_obsrm_pmpm_ct 560
rwjf_mental_distress_pct 882415
cons_chva 1000
rx_nonotc_dist_gpi6_pmpm_ct 2519
atlas_dirsales_farms12 604
cons_nwperadult 825
cms_tot_partd_payment_amt 53912
total_outpatient_allowed_pmpm_cost_6to9m_b4 143093
atlas_pct_sbp15 26326
rx_overall_mbr_resp_pmpm_cost 268313
rx_gpi2_66_pmpm_ct 1068
rx_gpi2_34_pmpm_ct 1445
atlas_pct_free_lunch14 853485
rx_overall_gpi_pmpm_ct_0to3m_b4 294
atlas_netmigrationrate1016 901460
rwjf_uninsured_pct 894775
phy_em_px_pct 159737
atlas_pct_laccess_seniors15 893676
rx_bh_mbr_resp_pmpm_cost_9to12m_b4 22079
atlas_berry_farms12 244
atlas_pct_laccess_hhnv15 897751
lab_dist_loinc_pmpm_ct 4510
atlas_me

({dtype('float64'): 169, dtype('int64'): 5},
 {('atlas_agritrsm_rct12', dtype('float64')),
  ('atlas_berry_acrespth12', dtype('float64')),
  ('atlas_berry_farms12', dtype('float64')),
  ('atlas_convspth14', dtype('float64')),
  ('atlas_dirsales_farms12', dtype('float64')),
  ('atlas_ffrpth14', dtype('float64')),
  ('atlas_fmrktpth16', dtype('float64')),
  ('atlas_freshveg_farms12', dtype('float64')),
  ('atlas_fsrpth14', dtype('float64')),
  ('atlas_ghveg_sqftpth12', dtype('float64')),
  ('atlas_grocpth14', dtype('float64')),
  ('atlas_medhhinc', dtype('float64')),
  ('atlas_naturalchangerate1016', dtype('float64')),
  ('atlas_net_international_migration_rate', dtype('float64')),
  ('atlas_netmigrationrate1016', dtype('float64')),
  ('atlas_orchard_acrespth12', dtype('float64')),
  ('atlas_orchard_farms12', dtype('float64')),
  ('atlas_pc_dirsales12', dtype('float64')),
  ('atlas_pc_ffrsales12', dtype('float64')),
  ('atlas_pc_fsrsales12', dtype('float64')),
  ('atlas_pc_snapben15', dt

In [7]:
dtypes = {}
isnullt = set()
for i in reg_cols:
    dtype = tdf[i].dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += 1
    if tdf[i].isnull().sum() > 0:
        isnullt.add((i, dtype))
    if dtype == 'float64': print(i, tdf[i].nunique())
dtypes, isnullt, isnullt - isnull

credit_bal_bankcard_severederog 522703
atlas_pc_fsrsales12 12759
atlas_pc_dirsales12 478193
rx_gpi2_01_pmpm_cost_0to3m_b4 6714
rx_gpi4_3400_pmpm_ct 959
rwjf_mv_deaths_rate 474876
atlas_fmrktpth16 430912
rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4 39481
rx_generic_pmpm_ct_0to3m_b4 203
atlas_orchard_acrespth12 413772
atlas_redemp_snaps16 480198
rev_pm_obsrm_pmpm_ct 404
rwjf_mental_distress_pct 475750
cons_chva 1000
rx_nonotc_dist_gpi6_pmpm_ct 1740
atlas_dirsales_farms12 574
cons_nwperadult 648
cms_tot_partd_payment_amt 49676
total_outpatient_allowed_pmpm_cost_6to9m_b4 89534
atlas_pct_sbp15 15020
rx_overall_mbr_resp_pmpm_cost 182238
rx_gpi2_66_pmpm_ct 681
rx_gpi2_34_pmpm_ct 966
atlas_pct_free_lunch14 460123
rx_overall_gpi_pmpm_ct_0to3m_b4 248
atlas_netmigrationrate1016 486161
rwjf_uninsured_pct 482416
phy_em_px_pct 91921
atlas_pct_laccess_seniors15 481770
rx_bh_mbr_resp_pmpm_cost_9to12m_b4 15985
atlas_berry_farms12 213
atlas_pct_laccess_hhnv15 484055
lab_dist_loinc_pmpm_ct 3007
atlas_medhhin

({dtype('float64'): 168, dtype('int64'): 6},
 {('atlas_agritrsm_rct12', dtype('float64')),
  ('atlas_berry_acrespth12', dtype('float64')),
  ('atlas_berry_farms12', dtype('float64')),
  ('atlas_convspth14', dtype('float64')),
  ('atlas_dirsales_farms12', dtype('float64')),
  ('atlas_ffrpth14', dtype('float64')),
  ('atlas_fmrktpth16', dtype('float64')),
  ('atlas_freshveg_farms12', dtype('float64')),
  ('atlas_fsrpth14', dtype('float64')),
  ('atlas_ghveg_sqftpth12', dtype('float64')),
  ('atlas_grocpth14', dtype('float64')),
  ('atlas_medhhinc', dtype('float64')),
  ('atlas_naturalchangerate1016', dtype('float64')),
  ('atlas_net_international_migration_rate', dtype('float64')),
  ('atlas_netmigrationrate1016', dtype('float64')),
  ('atlas_orchard_acrespth12', dtype('float64')),
  ('atlas_orchard_farms12', dtype('float64')),
  ('atlas_pc_dirsales12', dtype('float64')),
  ('atlas_pc_ffrsales12', dtype('float64')),
  ('atlas_pc_fsrsales12', dtype('float64')),
  ('atlas_pc_snapben15', dt

In [8]:
for i in tqdm(cat_cols):
    dtype = df[i].dtype
    assert dtype == tdf[i].dtype or i in ['auth_3mth_acute_mus', 'auth_3mth_acute_sns', 'auth_3mth_acute_cir', 'auth_3mth_snf_direct', 'auth_3mth_acute_cer', 'auth_3mth_dc_custodial', 'auth_3mth_dc_left_ama', 'auth_3mth_transplant', 'auth_3mth_post_acute_men', 'auth_3mth_post_acute_sns', 'auth_3mth_acute_chf', 'auth_3mth_acute_skn', 'auth_3mth_acute_res', 'auth_3mth_post_acute_mus']
    ie = LabelEncoder()
    if i in ['auth_3mth_snf_direct', 'auth_3mth_post_acute_sns', 'auth_3mth_post_acute_mus', 'auth_3mth_post_acute_men']:
        df[i] = df[i].apply(lambda x: 0).astype('int64')
        df[i] = ie.fit_transform(df[i])
        tdf[i] = ie.transform(tdf[i])
    if i in ['auth_3mth_acute_mus', 'auth_3mth_dc_custodial', 'auth_3mth_dc_left_ama', 'auth_3mth_transplant', 'auth_3mth_acute_res', 'auth_3mth_acute_chf']:
        df[i] = df[i].apply(lambda x: 1 if x  == '1' else 0).astype('int64')
        df[i] = ie.fit_transform(df[i])
        tdf[i] = ie.transform(tdf[i])
    elif i in ['auth_3mth_acute_sns', 'auth_3mth_acute_cir', 'auth_3mth_acute_skn', 'auth_3mth_acute_cer']:
        tdf[i] = tdf[i].apply(lambda x: 1 if x  == '1' else 0).astype('int64')
        df[i] = ie.fit_transform(df[i])
        tdf[i] = ie.transform(tdf[i])
    elif dtype == 'float64':    
        df[i] = ie.fit_transform(df[i].fillna(0.))
        tdf[i] = ie.transform(tdf[i].fillna(0.))
    elif dtype == 'O':    
        df[i] = ie.fit_transform(df[i].fillna('nan'))
        tdf[i] = ie.transform(tdf[i].fillna('nan'))
    else:
        df[i] = ie.fit_transform(df[i])
        tdf[i] = ie.transform(tdf[i])

  0%|          | 0/191 [00:00<?, ?it/s]

In [9]:
for i in tqdm(reg_cols):
    dtype = df[i].dtype
    assert dtype == tdf[i].dtype or i in ['auth_3mth_post_acute_mean_los']
    if i == 'auth_3mth_post_acute_mean_los':
        tdf[i] = np.float64(tdf[i])
    elif dtype == 'float64':    
        df[i] = df[i].fillna(0.)
        tdf[i] = tdf[i].fillna(0.)
    elif dtype == 'O':
        assert False
#         df[i] = ie.fit_transform(df[i].fillna('nan'))
#         tdf[i] = ie.transform(tdf[i].fillna('nan'))
    elif dtype == 'int64':
        pass
#         df[i] = ie.fit_transform(df[i])
#         tdf[i] = ie.transform(tdf[i])

  0%|          | 0/174 [00:00<?, ?it/s]

In [10]:
target_map = {"no_vacc": 0, 'vacc': 1}
df[target] = df[target].apply(lambda x: target_map[x]).astype(np.int64)
# shd not be any null value
sum(df.isnull().sum() > 0), sum(tdf.isnull().sum() > 0)


(0, 0)

In [11]:
# debug

In [12]:
[i for i in cat_cols + reg_cols if df[i].dtype != tdf[i].dtype]

[]

In [13]:
constant_filter = VarianceThreshold(threshold=.1)
constant_filter.fit(df[reg_cols])
constant_columns = [column for column in reg_cols if column not in df[reg_cols].columns[constant_filter.get_support()]]
reg_cols = list(set(reg_cols) - set(constant_columns))

In [14]:
X1, X2, y = df[reg_cols], df[cat_cols], df[target]

In [15]:
cat_cols

['rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4',
 'total_physician_office_visit_ct_pmpm_t_6-3-0m_b4',
 'cons_hhcomp',
 'auth_3mth_dc_custodial',
 'rx_gpi2_62_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_bld',
 'rx_maint_pmpm_ct_t_6-3-0m_b4',
 'auth_3mth_post_acute_cad',
 'auth_3mth_post_acute_vco',
 'rx_nonbh_pmpm_ct_t_9-6-3m_b4',
 'auth_3mth_post_acute_mus',
 'auth_3mth_post_acute_gus',
 'rx_gpi2_33_pmpm_ct_0to3m_b4',
 'rx_maint_pmpm_cost_t_12-9-6m_b4',
 'bh_ip_snf_net_paid_pmpm_cost_6to9m_b4',
 'cons_hxwearbl',
 'rx_nonotc_pmpm_cost_t_6-3-0m_b4',
 'auth_3mth_post_acute_res',
 'atlas_foodinsec_13_15',
 'rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
 'med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_vco',
 'rev_cms_ansth_pmpm_ct',
 'lang_spoken_cd',
 'auth_3mth_bh_acute_mean_los',
 'auth_3mth_post_acute_sns',
 'total_allowed_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_inf',
 'cons_rxadhm',
 'total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_ccs_048',
 'auth_3

In [16]:
reg_cols

['credit_bal_bankcard_severederog',
 'atlas_pc_fsrsales12',
 'atlas_pc_dirsales12',
 'rx_gpi2_01_pmpm_cost_0to3m_b4',
 'rwjf_mv_deaths_rate',
 'rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4',
 'rx_generic_pmpm_ct_0to3m_b4',
 'atlas_orchard_acrespth12',
 'atlas_redemp_snaps16',
 'cons_chva',
 'rx_nonotc_dist_gpi6_pmpm_ct',
 'atlas_dirsales_farms12',
 'cons_nwperadult',
 'cms_tot_partd_payment_amt',
 'total_outpatient_allowed_pmpm_cost_6to9m_b4',
 'atlas_pct_sbp15',
 'rx_overall_mbr_resp_pmpm_cost',
 'atlas_pct_free_lunch14',
 'rx_overall_gpi_pmpm_ct_0to3m_b4',
 'atlas_netmigrationrate1016',
 'phy_em_px_pct',
 'atlas_pct_laccess_seniors15',
 'rx_bh_mbr_resp_pmpm_cost_9to12m_b4',
 'atlas_berry_farms12',
 'atlas_pct_laccess_hhnv15',
 'lab_dist_loinc_pmpm_ct',
 'atlas_medhhinc',
 'auth_3mth_post_acute_mean_los',
 'atlas_povertyunder18pct',
 'atlas_pct_fmrkt_wiccash16',
 'credit_bal_autobank',
 'atlas_pct_fmrkt_snap16',
 'credit_bal_autofinance_new',
 'rx_overall_mbr_resp_pmpm_cost_0to3m_b4',
 'a

In [17]:
len(cat_cols), len(reg_cols)

(191, 139)

In [18]:
df.to_csv('dataset/transformed_dataset.csv', index=False, columns=[target] + cat_cols + reg_cols)
tdf.to_csv('dataset/transformed_dataset_holdout.csv', index=False, columns=cat_cols + reg_cols + [id])